In [3]:
import geopandas as gpd

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [ ]:
import json

In [15]:
dfs = [pd.read_csv(f'../data/GSAP_data/Data_{year}.csv') for year in [2010, 2019,2021]]
all_df = pd.concat(dfs).set_index('geo_code')

In [22]:
# Group by geo_code and aggregate columns into lists
result = all_df.groupby(all_df.index).agg(list)
result['code'] = result.code.apply(lambda x: x[0])

In [50]:
nested_dict = result.reset_index().groupby('code').apply(
    lambda x: x.set_index('geo_code')[output_keys].to_dict(orient='index'),
    include_groups=False ## don't include the column w/ the group keys when applying to the group by
).to_dict()

with open('../app/public/data/gsap.json','w') as handle:
    json.dump(nested_dict,handle)

## Output the geometry

In [5]:
gsap_2021_gdf = gpd.read_file('../data/GSAP_geom/GSAP_AM24_2021.shp')

In [28]:
for group, data in gsap_2021_gdf.groupby('code'):
    base = '../app/public/data/sub-national-geometry/'
    data = data[['geo_code','geo_name','geometry']].copy()
    data['geometry'] = data['geometry'].simplify(tolerance=0.001, preserve_topology=True)
    data.to_file(os.path.join(base,f"{group}_gsap_geometry.geojson"),driver='GeoJSON')